In [325]:
import pandas as pd
import numpy as np
import glob
import os
import pydotplus
from sklearn import tree
from IPython.display import Image, display
from statistics import mean
from sklearn.metrics import accuracy_score

In [274]:
# compare with other files and think about deleting 'IsUsedBGAdjust' - here everywhere was 0
def before_training(array_with_files):
    X = []
    Y = []
    for frame in array_with_files:
        Y.append(frame['Type'].astype({'Type':'category'}))
        fr = frame.drop(['Type', 'SystematicName', 'Description', 
                         'PositionX', 'PositionY', 'ErrorModel' ], axis=1)
        fr = fr.astype({'FeatureNum':'int32', 
             'Row':'int32', 
             'Col':'int32', 
             'accessions':'category', 
             'chr_coord':'category', 
             'SubTypeMask':'int32',
             'SubTypeName':'category', 
             'ProbeUID':'int32', 
             'ControlType':'category', 
             'ProbeName':'category', 
             'GeneName' : 'category',
             'gSurrogateUsed':'float', 
             'gIsFound': 'category', 
             'gProcessedSignal':'float', 
             'gProcessedSigError':'float',
             'gNumPixOLHi':'int32', 
             'gNumPixOLLo':'int32', 
             'gNumPix':'int32', 
             'gMeanSignal':'float', 
             'gMedianSignal':'float',
             'gPixSDev':'float', 
             'gPixNormIQR':'float', 
             'gBGNumPix':'int32', 
             'gBGMeanSignal':'float',
             'gBGMedianSignal':'float', 
             'gBGPixSDev':'float', 
             'gBGPixNormIQR':'float', 
             'gNumSatPix':'int32',
             'gIsSaturated':'category', 
             'gIsFeatNonUnifOL':'category', 
             'gIsBGNonUnifOL':'category', 
             'gIsFeatPopnOL':'category',
             'gIsBGPopnOL':'category', 
             'IsManualFlag':'category', 
             'gBGSubSignal':'float', 
             'gBGSubSigError':'float',
             'gIsPosAndSignif':'category', 
             'gPValFeatEqBG':'float', 
             'gNumBGUsed':'category', 
             'gIsWellAboveBG':'category',
             'gBGUsed':'float', 
             'gBGSDUsed':'float', 
             'gSpatialDetrendIsInFilteredSet':'category',
             'gSpatialDetrendSurfaceValue':'float', 
             'SpotExtentX':'float', 
             'SpotExtentY':'float',
             'gNetSignal':'float', 
             'gMultDetrendSignal':'float', 
             'gProcessedBackground':'float',
             'gProcessedBkngError':'float', 
             'IsUsedBGAdjust': 'category', 
             'gInterpolatedNegCtrlSub':'float',
             'gIsInNegCtrlRange':'float', 
             'gIsUsedInMD\n': 'category' 
            })
        X.append(fr)
    
    return X, Y


def train_model(data, target):
    clf = tree.DecisionTreeClassifier(max_depth=2)
    clf = clf.fit(data, target)
    return clf

In [2]:
path ='/Users/polinaturiseva/Downloads/GSE26728_RAW'

In [3]:
#  low (TDI) and high (NOAEL)
# this upload files
os.chdir(r'/Users/polinaturiseva/Downloads/GSE26728_RAW')
mice = glob.glob('*.txt')

In [191]:
def upload_all_files(array_with_names):
    data = []
    for name in array_with_names:
        data.append(upload_file(name))
    
    return data


def upload_file(name):
    print(f"File {name} started")
    df = pd.DataFrame()
    f = open(f"{path}/{name}", "r")
    for i in range(10):
        s = f.readline()
    labels = s.split('\t')
    df = pd.DataFrame(columns=labels)
    s = f.readline()
    loc =pd.DataFrame([s.split('\t')], columns=labels)
    intermediate =pd.DataFrame(columns=labels)
    i = 0
    while s!='':
        loc =pd.DataFrame([s.split('\t')], columns=labels)
        i += 1
        if i%5000==0:
#             print(i)
            df = pd.concat([df,intermediate,loc])
            intermediate =pd.DataFrame(columns=labels)
        else:
            intermediate = intermediate.append(loc)
        
        s = f.readline()
    
    if len(intermediate)!=0:
        df = pd.concat([df,intermediate])

    f.close()
    
    if len(df)==len(df.drop_duplicates()):
        print("There were no duplicated rows in the file")
    else:
        print(f"There were {len(df)-len(df.drop_duplicates())} duplicated rows in the file")
    
    if len(df)==len(df['GeneName'].unique()):
        print("there are no gene duplicates")
    else:
        print(f"There are gene duolicates.Only {len(df['GeneName'].unique())} are unique.")
        
    df = df.drop(['FEATURES', 'SystematicName', 'Description', 'PositionX', 'PositionY','ErrorModel' ], axis=1)
    if 'TDI' in name:
        df['Type'] ='low'
    else:
        if 'NOAEL' in name:
            df['Type'] ='high'
        else:
            df['Type'] ='control'
            
    print(f"File {name} finished")        
    return df


def check_for_gene_duplications_inside(array_with_data):
    
    combined = pd.concat(array_with_data)
    if len(combined)==len(combined.drop_duplicates()):
        print("There were no duplicated rows between files")
    else:
        print(f"There were {len(combined)-len(combined.drop_duplicates())} duplicated rows between files")
        
    only_genes = []
    control = pd.DataFrame(columns=['GeneName', 'gMeanSignal'])
    low  = control
    high = control
    
    for frame in array_with_data:
        only_genes.append(frame['GeneName'].unique())
        if frame['Type'].iloc[0] =='high':
            high = low.append(frame[['GeneName', 'gMeanSignal']])
        elif frame['Type'].iloc[0]=='low':
            low = low.append(frame[['GeneName', 'gMeanSignal']])
        else:
            control = control.append(frame[['GeneName', 'gMeanSignal']])
    
    
    
    control = control.groupby('GeneName')['gMeanSignal'].apply(lambda tags: mean(pd.to_numeric(tags)))
    high = high.groupby('GeneName')['gMeanSignal'].apply(lambda tags: mean(pd.to_numeric(tags)))
    low = low.groupby('GeneName')['gMeanSignal'].apply(lambda tags: mean(pd.to_numeric(tags)))
    
    print("control average is \n",   control)
    print("low dose average is \n",  low)
    print("high dose average is \n", high)
    
    prev = only_genes[0]
    for cur in only_genes[1:]:
        prev = dataframe_difference(prev, cur)
        
    print(f"{len(prev)} genes are analyzed in all of the recordings")
    
    return control, high, low
    
    
def dataframe_difference(list1, list2):
    """Find rows which genes are present in both dataframes"""
    return list(set(list1).intersection(list2))

In [326]:
all_files = upload_all_files(mice)
copy_all_files = all_files
control, high, low = check_for_gene_duplications_inside(all_files)
control.to_csv('/Users/polinaturiseva/Desktop/studying/jetBrains/control.csv')
high.to_csv('/Users/polinaturiseva/Desktop/studying/jetBrains/high.csv')
low.to_csv('/Users/polinaturiseva/Desktop/studying/jetBrains/low.csv')

File GSM658081_TDI1.txt started
There were no duplicated rows in the file
There are gene duolicates.Only 27935 are unique.
File GSM658081_TDI1.txt finished
File GSM658092_NOAEL6.txt started
There were no duplicated rows in the file
There are gene duolicates.Only 27935 are unique.
File GSM658092_NOAEL6.txt finished
File GSM658080_C6.txt started
There were no duplicated rows in the file
There are gene duolicates.Only 27935 are unique.
File GSM658080_C6.txt finished
File GSM658079_C5.txt started
There were no duplicated rows in the file
There are gene duolicates.Only 27935 are unique.
File GSM658079_C5.txt finished
File GSM658090_NOAEL4.txt started
There were no duplicated rows in the file
There are gene duolicates.Only 27935 are unique.
File GSM658090_NOAEL4.txt finished
File GSM658086_TDI6.txt started
There were no duplicated rows in the file
There are gene duolicates.Only 27935 are unique.
File GSM658086_TDI6.txt finished
File GSM658077_C3.txt started
There were no duplicated rows in t

27935 genes are analyzed in all of the recordings


In [193]:
control, high, low = check_for_gene_duplications_inside(all_files)

There were no duplicated rows between files
control average is 
 GeneName
0610005C13Rik     5037.088667
0610006I08Rik     2949.894167
0610007C21Rik     5547.365167
0610007L01Rik     2121.787150
0610007N19Rik      217.311067
0610007P06Rik     1004.819317
0610007P08Rik       47.998122
0610007P14Rik    10809.393500
0610007P22Rik     1196.858683
0610008C08Rik     1439.757552
0610008F07Rik      449.624233
0610009B14Rik       43.763353
0610009B22Rik     1606.727350
0610009D07Rik     2146.578319
0610009J22Rik     1874.057000
0610009K11Rik      793.971311
0610009K14Rik       39.634940
0610009O03Rik      227.169067
0610009O20Rik      141.372646
0610010D20Rik     2980.761167
0610010D24Rik       67.689788
0610010E21Rik     1310.840200
0610010F05Rik       98.434301
0610010K06Rik      347.240017
0610010K14Rik      409.223583
0610010O12Rik       68.061585
0610011F06Rik     6635.383000
0610011L14Rik       88.563011
0610012D14Rik      333.556433
0610012D17Rik      337.621867
                     ...  

In [205]:
control.to_csv('/Users/polinaturiseva/Desktop/studying/jetBrains/control.csv')
high.to_csv('/Users/polinaturiseva/Desktop/studying/jetBrains/high.csv')
low.to_csv('/Users/polinaturiseva/Desktop/studying/jetBrains/low.csv')

 **Результаты**  
 
 * 27935 genes are analyzed
 * все строки уникальны, то есть полных дупликатов нет, но есть дупликаты по генам, в том плане, что в колонке "GeneName" есть повторы
 * повторы распределены неравномерно
 * средние значения по категориям доступны в приложенных файлах
 

In [275]:
X, Y =before_training(all_files)

In [305]:
X_concat = pd.DataFrame() 
Y_concat = pd.DataFrame() 
X_concat = pd.concat(X)
Y_concat = pd.concat(Y).to_frame()

cat_columns = X_concat.select_dtypes(['category']).columns
X_concat[cat_columns] = X_concat[cat_columns].apply(lambda x: x.cat.codes)

cat_columns = Y_concat.select_dtypes(['category']).columns
Y_concat[cat_columns] = Y_concat[cat_columns].apply(lambda x: x.cat.codes)

In [306]:
model = train_model(X_concat, Y_concat)

In [321]:
for name, importance in zip(X_concat.columns, model.feature_importances_):
    if importance > 0:
        print(name, importance)

gSpatialDetrendSurfaceValue 0.8671674108605733
gMultDetrendSignal 0.1328325891394267


In [323]:
from sklearn.metrics import accuracy_score

In [324]:
y_pred = model.predict(X_concat)
accuracy_score(y_pred, Y_concat)

0.4736981750509673

**Заметки по дереву**  

Я знаю, что выполнила задание некорректно, поздно села. Поэтому я напишу здесь мысли, которые не успела реализовать.   
* По идее данные по генам надо было нормализовать (от минимума до максимума здесь некорректно, а вот стандартизовать значения для генов среди всех мышей было бы лучше)
* По хорошему учить модель надо не построчно, а на таблицах, так как признаком может быть скореллированность экспрессий от каких-то двух генов или что-то вроде того
